In [1]:
import os, pprint, shutil
from ordered_set import OrderedSet

# template_api_path = "../sample_files/base-template/"
# source_api_path = "../sample_files/api-current/"
# destination_api_path = "../sample_files/api-destination/"

template_api_path = "../api_data/api_template/"
source_api_path = "../api_data/api_existing/"
destination_api_path = "../api_data/api_converted/"

filename = "build.gradle"
template_file = os.path.join(template_api_path, filename)
source_file = os.path.join(source_api_path, filename)
destination_file = os.path.join(destination_api_path, filename)

try:
    #if path already exists, remove it before copying with copytree()
    if os.path.exists(destination_api_path):
        shutil.rmtree(destination_api_path)
        shutil.copytree(source_api_path, destination_api_path)
except OSError as e:
    # If the error was caused because the source wasn't a directory
    if e.errno == errno.ENOTDIR:
        shutil.copy(source_api_path, destination_api_path)
    else:
        print('Directory not copied. Error: %s' % e)

In [2]:
# with open(source_file) as file:
#     for line in file:
#         print(line.rstrip())

In [3]:
with open(template_file, "r") as f, open(source_file, "r") as out:
    template_build_line_start = 0
    template_build_line_end = 0
    template_dep_line_start = 0
    template_dep_line_end = 0
    for index, line in enumerate(f):
        if line.startswith("buildscript {"):
            template_build_line_start = index
        if line.startswith("\tdependencies {"):
            template_dep_line_start = index
        if template_dep_line_start !=0 and line.startswith("\t}"):
            template_dep_line_end = index
        if template_build_line_start !=0 and line.startswith("}"):
            template_build_line_end = index
            break

    source_build_line_start = 0
    source_build_line_end = 0
    source_dep_line_start = 0
    source_dep_line_end = 0
    for index_out, line_out in enumerate(out):
        if line_out.startswith("buildscript {"):
            source_build_line_start = index_out
        if line_out.startswith("\tdependencies {"):
            source_dep_line_start = index_out
        if source_dep_line_start !=0 and line_out.startswith("\t}"):
            source_dep_line_end = index_out
        if source_build_line_start !=0 and line_out.startswith("}"):
            source_build_line_end = index_out
            break

In [4]:
with open(template_file, "r") as f, open(source_file, "r") as out:
    template_line_start = 0
    template_line_end = 0
    for index, line in enumerate(f):
        if line.startswith("dependencies {"):
            template_line_start = index
        if template_line_start !=0 and line.startswith("}"):
            template_line_end = index
            break

    source_line_start = 0
    source_line_end = 0
    for index_out, line_out in enumerate(out):
        if line_out.startswith("dependencies {"):
            source_line_start = index_out
        if source_line_start !=0 and line_out.startswith("}"):
            source_line_end = index_out
            break

In [5]:
with open(template_file, "r") as f:
    i = 0
    template_build_dict_list = []
    template_build_list = []
    for index, line in enumerate(f):
        if index > template_dep_line_start and index < template_dep_line_end and line !='\n':
            template_build_list.append(line)
            data = {}
            line_data = line.strip().replace("'", "").replace(" ", ":").split(":")
#             print("Line {}: {}".format(index, line_data))
            i += 1
            if i == 2:
                if len(line_data) > 1:
                    ver_list = line_data[len(line_data)-1].split('.')
                    ver_list = [int(x) for x in ver_list]

                    data["prefix"] = line_data[0]
                    data["group"] = line_data[len(line_data)-3]
                    data["artifact"] = line_data[len(line_data)-2]
                    sufix = line_data[len(line_data)-2].split('-')
                    data["artifact_suffix"] = sufix[len(sufix)-1]
                    data["version"] = line_data[len(line_data)-1]
                    data["version_sum"] = sum(ver_list)
                    data['template_line'] = line
                    data['rule'] = 1
            elif i == 3:
                if len(line_data) > 1:
                    ver_list = line_data[len(line_data)-1].replace("-", ".").split(".")
                    ver_list = [int(x) for x in ver_list[:-1]]

                    data["prefix"] = line_data[0]
                    data["group"] = line_data[len(line_data)-3]
                    data["artifact"] = line_data[len(line_data)-2]
                    sufix = line_data[len(line_data)-2].split('-')
                    data["artifact_suffix"] = sufix[len(sufix)-1]
                    data["version"] = line_data[len(line_data)-1]
                    data["version_sum"] = sum(ver_list)
                    data['template_line'] = line
                    data['rule'] = 2
            else:
                if len(line_data) > 1:
                    data["prefix"] = line_data[0]
                    data["group"] = line_data[len(line_data)-2]
                    data["artifact"] = line_data[len(line_data)-1]
                    sufix = line_data[len(line_data)-1].split('-')
                    data["artifact_suffix"] = sufix[len(sufix)-1]
                    data["version"] = None
                    data["version_sum"] = 0
                    data['template_line'] = line
                    data['rule'] = None
            template_build_dict_list.append(data)

# pprint.pprint(template_build_dict_list)
# pprint.pprint(template_build_list)

In [6]:
with open(template_file, "r") as f:
    i = 0
    template_dict_list = []
    template_list = []
    for index, line in enumerate(f):
        if index > template_line_start and index < template_line_end and line !='\n':
            template_list.append(line)
            data = {}
            template_split_line = line.strip().replace("'", "").replace("ion ", "ion:").replace(" ", "")
#             print("Line {}: {}".format(index, template_split_line))
            i += 1
            if i == 1 or i == 8 or i >= 13:
                line_data = template_split_line.split(":")
                if len(line_data) > 1:
                    ver_list = line_data[len(line_data)-1].split('.')
                    ver_list = [int(x) for x in ver_list]

                    data["prefix"] = line_data[0]
                    data["group"] = line_data[len(line_data)-3]
                    data["artifact"] = line_data[len(line_data)-2]
                    sufix = line_data[len(line_data)-2].split('-')
                    data["artifact_suffix"] = sufix[len(sufix)-1]
                    data["version"] = line_data[len(line_data)-1]
                    data["version_sum"] = sum(ver_list)
                    data['template_line'] = line
                    data['rule'] = 1
            elif i == 10:
                line_data = template_split_line.replace(",", ":").split(":")
                if len(line_data) > 1:
                    ver_list = line_data[len(line_data)-1].split('.')
                    ver_list = [int(x) for x in ver_list]

                    data["prefix"] = line_data[0]
                    data["group"] = line_data[len(line_data)-5]
                    data["artifact"] = line_data[len(line_data)-3]
                    data["artifact_suffix"] = line_data[len(line_data)-3]
                    data["version"] = line_data[len(line_data)-1]
                    data["version_sum"] = sum(ver_list)
                    data['template_line'] = line
                    data['rule'] = 2
            else:
                line_data = template_split_line.split(":")
                if len(line_data) > 1:
                    data["prefix"] = line_data[0]
                    data["group"] = line_data[len(line_data)-2]
                    data["artifact"] = line_data[len(line_data)-1]
                    sufix = line_data[len(line_data)-1].split('-')
                    data["artifact_suffix"] = sufix[len(sufix)-1]
                    data["version"] = None
                    data["version_sum"] = 0
                    data['template_line'] = line
                    data['rule'] = None
            template_dict_list.append(data)

# pprint.pprint(template_dict_list)
# pprint.pprint(template_list)

In [7]:
with open(source_file, "r") as f:
    template_build_set = OrderedSet()
    unchange_build_set = OrderedSet()
    change_build_set = OrderedSet()
    for index_out, line_out in enumerate(f):
        if index_out > source_dep_line_start and index_out < source_dep_line_end:
#             print("Line {}: {}".format(index_out, line_out.strip()))
            line_data = line_out.strip().replace("'", "").replace(" ", ":").split(":")
#             print("Line {}: {}".format(index_out, line_data))
#             line_data = template_split_line.replace(",", ":").split(":")
#             print("Line {}: {}".format(index_out, line_data))
            for template in template_build_dict_list:
#                 template_build_set.add(template['template_line'])
                if len(line_data) > 2:
                    if template['group'] == line_data[1]:
                        if template['artifact'] == line_data[len(line_data)-2]:
                            template_build_set.add(template['template_line'])
                            change_build_set.add(line_out)
                        elif template['artifact'] == line_data[len(line_data)-1]:
                            template_build_set.add(template['template_line'])
                            change_build_set.add(line_out)
                    elif template['group'] == line_data[2]:
                        if template['artifact'] == line_data[len(line_data)-3] and template['version'] == line_data[len(line_data)-1]:
                            template_build_set.add(template['template_line'])
                            change_build_set.add(line_out)
                    else:
                        unchange_build_set.add(line_out)
                else:
                    unchange_build_set.add(line_out)


In [8]:
with open(source_file, "r") as f:
    template_set = OrderedSet()
    unchange_set = OrderedSet()
    change_set = OrderedSet()
    for index_out, line_out in enumerate(f):
        if index_out > source_line_start and index_out < source_line_end:
#             print("Line {}: {}".format(index_out, line_out.strip()))
            template_split_line = line_out.strip().replace("'", "").replace("ion ", "ion:").replace("pile ", "pile:").replace(" ", "")
#             print("Line {}: {}".format(index_out, template_split_line))
            line_data = template_split_line.replace(",", ":").split(":")
#             print("Line {}: {}".format(index_out, line_data))
            for template in template_dict_list:
#                 template_set.add(template['template_line'])
                if len(line_data) > 2:
                    if template['group'] == line_data[1]:
                        if template['artifact'] == line_data[len(line_data)-2]:
                            template_set.add(template['template_line'])
                            change_set.add(line_out)
                        elif template['artifact'] == line_data[len(line_data)-1]:
                            template_set.add(template['template_line'])
                            change_set.add(line_out)
                    elif template['group'] == line_data[2]:
                        if template['artifact'] == line_data[len(line_data)-3] and template['version'] == line_data[len(line_data)-1]:
                            template_set.add(template['template_line'])
                            change_set.add(line_out)
                    else:
                        unchange_set.add(line_out)
                else:
                    unchange_set.add(line_out)


In [9]:
d = {}
for t in template_build_dict_list:
    for i, u in enumerate(unchange_build_set):
        if t['group'] in u and t['artifact_suffix'] in u:
            line_data = u.strip().replace("'", "").replace(" ", ":").split(":")
            if len(line_data) > 1 and t['rule'] == 1:
                ver_list = line_data[len(line_data)-1].split('.')
                if not ''.join(ver_list[-1:]).isdigit():
                    ver_list = ver_list[:-1]
                ver_list = [int(x) for x in ver_list]

                d["version"] = line_data[len(line_data)-1]
                d["version_sum"] = sum(ver_list)
                d['template_line'] = u

                if d["version_sum"] > t['version_sum']:
                    for tmp in template_build_set:
                        if t['group'] in tmp and t['artifact_suffix'] in tmp:
                            template_build_set.remove(tmp)
                            add_line = t['template_line'].replace(str(t['version']), str(d["version"]))
                            template_build_set.add(add_line)

diff_build = (unchange_build_set - change_build_set)
# pprint.pprint(diff_build)

In [10]:
d = {}
for t in template_dict_list:
    for i, u in enumerate(unchange_set):
        if t['group'] in u and t['artifact_suffix'] in u:
            template_split_line = u.strip().replace("'", "").replace("pile ", "pile:").replace(" ", "")
            line_data = template_split_line.split(":")
            if len(line_data) > 1 and t['rule'] == 1:
                ver_list = line_data[len(line_data)-1].split('.')
                ver_list = [int(x) for x in ver_list]

                d["version"] = line_data[len(line_data)-1]
                d["version_sum"] = sum(ver_list)
                d['template_line'] = u

                if d["version_sum"] > t['version_sum']:
                    for tmp in template_set:
                        if t['group'] in tmp and t['artifact_suffix'] in tmp:
                            template_set.remove(tmp)
                            add_line = t['template_line'].replace(str(t['version']), str(d["version"]))
                            template_set.add(add_line)

diff = (unchange_set - change_set)
# pprint.pprint(diff)

In [11]:
compose_build_template = []
compose_build_template.append(['\tdependencies {\n'])
compose_build_template.append(template_build_set)
compose_build_template.append(diff_build)
compose_build_template.append(['\t}\n'])

compose_build_template = [line for level in compose_build_template for line in level]
# pprint.pprint(compose_build_template)

with open(source_file, "r") as f:
    final_data_list = []
    final_build_data_list = []
    build_data_list = []
    for index_, line_ in enumerate(f):
        if index_ > source_build_line_start-1 and index_ < source_build_line_end+1:
            if index_ > source_dep_line_start-1 and index_ < source_dep_line_end+1:
                continue
            build_data_list.append(line_)
            continue
        if index_ > source_line_start-1 and index_ < source_line_end+1:
            continue
        final_data_list.append(line_)
        final_build_data_list.append(line_)
        
for line in compose_build_template:
    build_data_list.append(line)

build_data_list.remove("}\n")
build_data_list.append("}\n")

for line in build_data_list:
    final_build_data_list.append(line)

# pprint.pprint(build_data_list)

In [12]:
compose_template = []
compose_template.append(['\ndependencies {\n'])
compose_template.append(template_set)
compose_template.append(diff)
compose_template.append(['}\n\n'])

compose_template = [line.replace('compile', 'implementation').replace('testCompile', 'testImplementation') for level in compose_template for line in level]
# pprint.pprint(compose_template)

compose_template.append(build_data_list)
# pprint.pprint(compose_template)

for line in compose_template:
    final_data_list.append(line)

# pprint.pprint(final_data_list)

In [13]:
with open(destination_file, "w+") as f:
    for line in final_data_list:
        f.writelines(line)